In [ ]:
import pandas as pd

In [ ]:
fd_file = pd.read_excel(r"C:\Users\kaihsu\Desktop\財務發送對帳單.xlsx", header=7)

In [ ]:
fd_file = fd_file.dropna(subset=['INVOICE NO'])

In [ ]:
fd_file['發票日'] = pd.to_datetime(fd_file['發票日'], yearfirst=True)

In [ ]:
fd_file = fd_file[['發票日', 'INVOICE NO', '幣別', '金額']]

In [ ]:
fd_file = fd_file.groupby(['發票日', 'INVOICE NO', '幣別'])[['金額']].sum().reset_index()

In [ ]:
fd_file.columns = ['INVOICE DATE', 'INVOICE NO', 'CURRENCY', 'AMOUNT']

In [ ]:
fd_file['INVOICE DATE'] = fd_file['INVOICE DATE'].dt.strftime('%Y-%m-%d')

In [ ]:
axrr = pd.read_excel(r"C:\Users\kaihsu\Desktop\axrr330.xls", header=6)    

In [ ]:
axrr = axrr.dropna(subset=['商用發票號碼'])

In [ ]:
axrr = axrr[['帳款日期', '商用發票號碼', '幣別', '未沖金額']]

In [ ]:
axrr.columns = ['INVOICE DATE', 'INVOICE NO', 'CURRENCY', 'AMOUNT']

In [ ]:
axrr['INVOICE DATE'] = pd.to_datetime(axrr['INVOICE DATE'], yearfirst=True)

In [ ]:
axrr = axrr.groupby(['INVOICE DATE', 'INVOICE NO', 'CURRENCY'])[['AMOUNT']].sum().reset_index()

In [ ]:
axrr['INVOICE DATE'] = axrr['INVOICE DATE'].dt.strftime('%Y-%m-%d')

In [ ]:
axrr

## PDC


In [ ]:
df = pd.read_clipboard()

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df['採購數量'] = df.apply(lambda x: x['採購數量'] * 1000 if x['採購單位'] == 'KPC' else x['採購數量'], axis=1)

In [ ]:
df = df[['料號', '採購數量', '訂單號碼', '採購項次', '訂單交期']]

In [ ]:
df['佳邦料號'] = ''

In [ ]:
append_dict = {4: '客戶發票', 5: '客戶單號', 6: '客戶收貨日期', 7: '項次', 9: '約定交貨日'}
for k, v in append_dict.items():
    df.insert(k, v, value=None)
df['項次'] = df.groupby(['項次']).cumcount()+1

## 處理Shara的對帳單

In [ ]:
df = pd.read_excel('複本 axrr330 _Example.xls')

In [ ]:
cus_code = df.iloc[0:, 1].str.extract(r'([A-Z]{2}[0-9]+)').dropna()[0].tolist()

In [ ]:
cus_code = tuple(cus_code) # get customer code

In [ ]:
df['Sign'] = df[df.columns.tolist()[1]].str.startswith('類別')

In [ ]:
df['Sign'] = df['Sign'].map(lambda x: 'Header' if x == True else '' if pd.isna(x) else '')

In [ ]:
headers = tuple(df[df['Sign'].isin(['Header'])].index.tolist()) # get table headers index

In [ ]:
headers # 下一張表格-6等於上一張結尾

In [ ]:
columns = df.loc[5, :].tolist() # column names

In [ ]:
index_list = df.index.tolist() # get all index length

In [ ]:
df

In [ ]:
table_list = []
for i in range(0, len(headers)):
    if headers[i] != headers[-1]:
        result = df.loc[headers[i]: headers[i+1] - 6, :].reset_index(drop=True)
        result.columns = columns
        result.drop(0, inplace=True)
        result.dropna(subset=['類別'], inplace=True)
        result['類別'] = cus_code[i]
        result = result[['帳款日期', '類別', '應收款日', '商用發票號碼', '幣別', '未沖金額']]
        result['帳款日期'] = pd.to_datetime(result['帳款日期'], yearfirst=True).dt.strftime('%Y-%m-%d')
        result['應收款日'] = pd.to_datetime(result['應收款日'], yearfirst=True).dt.strftime('%Y-%m-%d')
#         result = result.groupby(['帳款日期', '類別', '應收款日', '商用發票號碼', '幣別'])[['未沖金額']].sum().reset_index()
        globals()['table' + str(i)] = result
        table_list.append(globals()['table' + str(i)])
        
    else:
        result = df.loc[headers[-1]:index_list[-4],].reset_index(drop=True)
        result.columns = columns
        result.drop(0, inplace=True)
        result.dropna(subset=['類別'], inplace=True)
        result['類別'] = cus_code[-1]
        result = result[['帳款日期', '類別', '應收款日', '商用發票號碼', '幣別', '未沖金額']]
        result['帳款日期'] = pd.to_datetime(result['帳款日期'], yearfirst=True).dt.strftime('%Y-%m-%d')
        result['應收款日'] = pd.to_datetime(result['應收款日'], yearfirst=True).dt.strftime('%Y-%m-%d')
#         result = result.groupby(['帳款日期', '類別', '應收款日', '商用發票號碼', '幣別'])[['未沖金額']].sum().reset_index()
        globals()['table' + str(i)] = result
        table_list.append(globals()['table' + str(i)])

In [ ]:
final = pd.concat(table_list)

In [ ]:
final

In [ ]:
final['未沖金額'].sum()